In [1]:
!nvidia-smi

Tue Dec 12 08:21:56 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.5/660.5 kB 4.2 MB/s eta 0:00:00


In [1]:
!pip install roboflow

In [2]:
# If you don't have ClearML and ClearML Agent installed then uncomment these line
!pip install clearml
!pip install clearml-agent

In [3]:
from IPython.display import display, Image
from IPython import display
display.clear_output()
import os
import json

import ultralytics
from ultralytics import YOLO
ultralytics.checks()

from roboflow import Roboflow

from clearml import Task

Ultralytics YOLOv8.0.227 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 27.2/78.2 GB disk)


In [4]:
! export MPLBACKEND=TkAg

In [5]:
#@title Insert your own Credentials

web_server = 'https://app.clear.ml'#@param {type:"string"}
api_server = 'https://api.clear.ml'#@param {type:"string"}
files_server = 'https://files.clear.ml'#@param {type:"string"}
with open("secret.json") as file:
    access_key = json.load(file)["clearml_access_key"]#@param {type:"string"}
    secret_key = json.load(file)["clearml_secret_key"]#@param {type:"string"}

Task.set_credentials(web_host=web_server,
                     api_host=api_server,
                     files_host=files_server,
                     key=access_key,
                     secret=secret_key
                     )

In [6]:
# Run the agent, you will need to stop it manually once your tas gets an id number
!clearml-agent daemon --queue default

Using environment access key CLEARML_API_ACCESS_KEY=HW5700GYW80WJGC764JJ
Using environment secret key CLEARML_API_SECRET_KEY=********
Current configuration (clearml_agent v1.6.1, location: None):
----------------------
sdk.storage.cache.default_base_dir = ~/.clearml/cache
sdk.storage.cache.size.min_free_bytes = 10GB
sdk.storage.direct_access.0.url = file://*
sdk.metrics.file_history_size = 100
sdk.metrics.matplotlib_untitled_history_size = 100
sdk.metrics.images.format = JPEG
sdk.metrics.images.quality = 87
sdk.metrics.images.subsampling = 0
sdk.metrics.tensorboard_single_series_per_graph = false
sdk.network.metrics.file_upload_threads = 4
sdk.network.metrics.file_upload_starvation_warning_sec = 120
sdk.network.iteration.max_retries_on_server_error = 5
sdk.network.iteration.retry_backoff_factor_sec = 10
sdk.aws.s3.key = 
sdk.aws.s3.region = 
sdk.aws.boto3.pool_connections = 512
sdk.aws.boto3.max_multipart_concurrency = 16
sdk.log.null_log_propagate = false
sdk.log.task_log_buffer_capac

In [7]:
# Load the data sets from roboflow
with open("secret.json") as file:
    roboflow_api_key = json.load(file)["roboflow_api_key"]
rf = Roboflow(api_key=roboflow_api_key)
project = rf.workspace("ariel-drabkin-tifqg").project("solar-panel-detector-imvoh")
dataset = project.version(1).download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.0.227, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to solar-panel-detection-1 in yolov8:: 100%|██████████| 40644/40644 [00:11<00:00, 3639.10it/s]


In [ ]:
# Train https://docs.ultralytics.com/modes/train/
# Before the start of the training make sure to update the data.yaml file with the correct dir of each set
path = "/content/solar-panel-detection-1/data.yaml"
project = "Solar-Panel-Detector"
name = "mosaic-augmentation"

!yolo task=detect mode=train model=yolov8n.pt data={path} project={project} name={name} mosaic=1.0 mixup=0.0 copy_paste=0.0 close_mosaic=0 epochs=20 imgsz=680 save_period=5 patience=3 plots=True

Ultralytics YOLOv8.0.227 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/solar-panel-detection-1/data.yaml, epochs=20, patience=3, batch=16, imgsz=680, save=True, save_period=5, cache=False, device=None, workers=8, project=Solar-Panel-Detector, name=mosaic-and-mixup-0.5-augmentation4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_